# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
import pickle

In [8]:
service = Service('C:/Users/khb16/Desktop/code/PROJECT/Recommandation/chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [ ]:
pages = [] # 크롤링 오류 확인 위해 page 수 기록
title = []
lyric = []
artist = []
genres = []
writer = []
rank = []
p = re.compile("'(\d+)'")
for page in range(6551,10001,50): # 1페이지만 추출(1,51,50), 발라드 gnrCode=GN0100, 댄스 gnrCode=GN0200
    url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D\
                =&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex={}".format(page) # 댄스 페이지
    driver.get(url)
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 한 페이지에 있는 50개의 songId 추출
    songid = [] # songid

    for i in range(50):
        data = soup.find_all("a", class_= "btn button_icons type03 song_info")[i].get('href')
        id_ = p.search(str(data)).group(1)
        songid.append(id_)
        f = soup.select('td span.rank')[i].text
        rank.append(f)

    # 해당 songId 입력 후 정보 추출
    for song in songid:
        lyric_url = 'https://www.melon.com/song/detail.htm?songId={}'.format(song)
        driver.get(lyric_url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        a = soup.select('div.song_name')[0].text[3:].strip() # 제목
        title.append(a)
        try:
            driver.find_element_by_css_selector(".button_more.arrow_d").click() # 가사
            ly = driver.find_element_by_css_selector('.lyric.on')
            b = ly.text
            lyric.append(b)
        except:
            lyric.append('가사없음')
        ar = driver.find_element_by_css_selector(".artist")
        c = ar.text
        artist.append(c)
        d = soup.select('dl.list dd')[2].text # 장르
        genres.append(d)
        try:
            e = soup.select('div.section_prdcr a.artist_name')[0].text # 작사가 1명
            writer.append(e)
        except:
            writer.append('작사가없음')
        pages.append(page)

<ipython-input-87-c55ba20805d8>:38: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(".button_more.arrow_d").click() # 가사
<ipython-input-87-c55ba20805d8>:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ly = driver.find_element_by_css_selector('.lyric.on')
<ipython-input-87-c55ba20805d8>:44: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ar = driver.find_element_by_css_selector(".artist")


In [86]:
col = ['pages','rank','title','lyric','artist','genres','writer']
songlist = pd.DataFrame([pages, rank,title, lyric, artist, genres, writer]).T
songlist.columns = col
songlist

,pages,rank,title,lyric,artist,genres,writer
0,6101,6101,SELFISH (Feat. 슬기 Of Red Velvet),어느새 사회화가된 것 같아\n아무도 Blah Blah\n한 것 같진 않은데\n남들이...,마마무 (Mamamoo),댄스,박우상
1,6101,6102,Mixtape#2,처음 모두 만나 인사 한날 기억나\n걸음마를 떼는 연습에\n우린 아기 엄마처럼\n이...,Stray Kids (스트레이 키즈),댄스,방찬 (3RACHA)
2,6101,6103,Your Gravity,높은 하늘보다 밝은 태양보다\n내게 웃는 니가 더 끌어당기기에\n비교할 수조차 없을...,업텐션 (UP10TION),댄스,zomay
3,6101,6104,SUNSET,저 서쪽 하늘에서 붉은빛이 질 때\n그림자의 크기도 배가되어 Times Yeah\n...,AB6IX (에이비식스),댄스,이대휘 (AB6IX)
4,6101,6105,응원 (Go For It!),가끔은 조용히 생각해 보곤 해\n어디쯤 와 있는 걸까\n내 맘에 그려온 눈부신 날들...,제이레빗(J Rabbit),"댄스, 게임",손민수 (DEVSISTERS)
...,...,...,...,...,...,...,...
495,None,6596,None,None,None,None,None
496,None,6597,None,None,None,None,None
497,None,6598,None,None,None,None,None
498,None,6599,None,None,None,None,None


In [78]:
songlist = songlist.head(450)
songlist

,pages,rank,title,lyric,artist,genres,writer
0,6101,6101,SELFISH (Feat. 슬기 Of Red Velvet),어느새 사회화가된 것 같아\n아무도 Blah Blah\n한 것 같진 않은데\n남들이...,마마무 (Mamamoo),댄스,박우상
1,6101,6102,Mixtape#2,처음 모두 만나 인사 한날 기억나\n걸음마를 떼는 연습에\n우린 아기 엄마처럼\n이...,Stray Kids (스트레이 키즈),댄스,방찬 (3RACHA)
2,6101,6103,Your Gravity,높은 하늘보다 밝은 태양보다\n내게 웃는 니가 더 끌어당기기에\n비교할 수조차 없을...,업텐션 (UP10TION),댄스,zomay
3,6101,6104,SUNSET,저 서쪽 하늘에서 붉은빛이 질 때\n그림자의 크기도 배가되어 Times Yeah\n...,AB6IX (에이비식스),댄스,이대휘 (AB6IX)
4,6101,6105,응원 (Go For It!),가끔은 조용히 생각해 보곤 해\n어디쯤 와 있는 걸까\n내 맘에 그려온 눈부신 날들...,제이레빗(J Rabbit),"댄스, 게임",손민수 (DEVSISTERS)
...,...,...,...,...,...,...,...
445,6501,6546,Wanna Be,둡둡두룹둡\n둡둡두룹둡\n둡둡두룹둡\nWanna be your girl friend...,여자친구 (GFRIEND),"댄스, 국내드라마",ZigZag Note
446,6501,6547,지금 이대로 (NC 피버뮤직),어제랑 다름없이 웃어주면 돼\n처음에 반가웠던 그 마음으로\n참 좋았었지 난 하루에...,러블리즈,댄스,김이나
447,6501,6548,Little More,이런 기분이 너무 오랜만이지\n참 이상한 일이야\n왜 난 너만 보면은\n자꾸 웃게 ...,보아 (BoA),댄스,보아 (BoA)
448,6501,6549,English Girl,A to Z 네 모든 걸 다 알고 싶어져\n1 2 3 마주치는 눈빛이 짜릿한 걸\n...,원어스 (ONEUS),댄스,이상호


In [83]:
songlist = songlist.tail(429)
songlist

,pages,rank,title,lyric,artist,genres,writer
21,6101,6122,바보같은 Story,바보 같은 Story\n널 향한 내 Fantasy\n눈을 감고 널 그리면\n진짜가 ...,J.Fla,댄스,J.Fla
22,6101,6123,Storybook,넌 잃어버린 구두를 찾아\n내 앞에 나타났어\n눈이 부신 Girl oh\n마법 같은...,뉴이스트,댄스,김희선
23,6101,6124,물 한잔,뚜루뚜루 yoohoo\n뚜루뚜루 yoohoo\n바쁘고 힘든 하루가 이렇게\n시작이 ...,B1A4,댄스,진영
24,6101,6125,Shirt,Black and white shirt\n깃 세우고\nPop ya collar an...,SUPER JUNIOR (슈퍼주니어),댄스,제이덥 (JDUB)
25,6101,6126,니가 처음이야 (Feat. 기리보이),Ay girl\n내 안에 내 안에 너를 갖고 싶어\nAy girl\n이렇게 이렇게 ...,장현승,댄스,블랙아이드 필승
...,...,...,...,...,...,...,...
445,6501,6546,Wanna Be,둡둡두룹둡\n둡둡두룹둡\n둡둡두룹둡\nWanna be your girl friend...,여자친구 (GFRIEND),"댄스, 국내드라마",ZigZag Note
446,6501,6547,지금 이대로 (NC 피버뮤직),어제랑 다름없이 웃어주면 돼\n처음에 반가웠던 그 마음으로\n참 좋았었지 난 하루에...,러블리즈,댄스,김이나
447,6501,6548,Little More,이런 기분이 너무 오랜만이지\n참 이상한 일이야\n왜 난 너만 보면은\n자꾸 웃게 ...,보아 (BoA),댄스,보아 (BoA)
448,6501,6549,English Girl,A to Z 네 모든 걸 다 알고 싶어져\n1 2 3 마주치는 눈빛이 짜릿한 걸\n...,원어스 (ONEUS),댄스,이상호


In [84]:
songlist.to_pickle('dance_6122-6550.pkl', protocol = 4) # 발라드는 ballad로 저장

In [219]:
songlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   pages   100 non-null    object
 1   title   100 non-null    object
 2   lyric   100 non-null    object
 3   artist  100 non-null    object
 4   genres  100 non-null    object
 5   writer  100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB
